In [12]:

import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import pairwise_distances

In [5]:
import os 
os.getcwd()

'/home/yangxiang/ML_Project/Model/User_Based_CF'

NMF:
    https://surprise.readthedocs.io/en/stable/FAQ.html#how-to-get-the-top-n-recommendations-for-each-user

https://medium.com/sfu-cspmp/recommendation-systems-user-based-collaborative-filtering-using-n-nearest-neighbors-bf7361dc24e0


https://towardsdatascience.com/how-to-build-a-memory-based-recommendation-system-using-python-surprise-55f3257b2cf4

The key difference of memory-based approach from the model-based techniques is that we are not learning any parameter using gradient descent (or any other optimization algorithm). The closest user or items are calculated only by using Cosine similarity or Pearson correlation coefficients, which are only based on arithmetic operations. 

# Memory Based

In [6]:
path='/home/yangxiang/ML_Project/yelp_dataset/'

In [7]:
business_df=pd.read_json(path+'yelp_academic_dataset_business.json',lines=True)

In [8]:
#user_df=pd.read_json(path+'yelp_academic_dataset_user.json',lines=True)

In [9]:
review_df=pd.read_json(path+'yelp_academic_dataset_review.json',lines=True)

KeyboardInterrupt: 

In [ ]:
business_df.shape

In [ ]:
#user_df.shape

In [ ]:
review_df.shape

In [ ]:
business_df['city'].value_counts()

In [ ]:
business_df['state'].value_counts()

In [ ]:
#choose only Las Vegas and create business_LV dataframe
business_LV_df=business_df[business_df['city']=='Las Vegas']

In [ ]:
business_LV_df.shape

In [ ]:
# Leave only business id and name column in business_LV dataframe
business_LV_df=business_LV_df[['business_id','name']]

In [ ]:
# Leave only user_id','stars','business_id' column in review_df dataframe
review_df=review_df[['user_id','stars','business_id']]

In [ ]:
#merge business_LV_df and reviews
UBCF_data=pd.merge(business_LV_df,review_df, how='left', on='business_id')

In [ ]:
#Randomly Select 80000 data 
UBCF_data=UBCF_data.sample(n=80000)

In [ ]:
#Export Data as csv file 
UBCF_data.to_csv(r'UBCF_data.csv', index = False)

In [13]:
#Read data
UBCF_data=pd.read_csv('//home/yangxiang/ML_Project/UBCF_data.csv')

In [14]:
UBCF_data.head(5)

,business_id,name,user_id,stars
0,Gh8L5vvZC7v8KtICjNYugw,Enso Tea Bar,gi10Atz0Y_FvCDFHY2cx9Q,3
1,O_6zhr9_r49cgw1r5nJIwA,Turnt Up Tours,YshOhshfgyD86PwzaTrq_A,5
2,q0rAq1YMqm44ku2ihk747Q,Surefire Pest Control,7oevgQOu2bJFU1n5oM7dOg,5
3,PaKG_CtfPbjWQUcdro81rw,Drai's After Hours,SoNbQ4PpFFGbd52pv3bKFQ,5
4,BcW7Z9lPmOB_8eS2lEuOqQ,Musashi Japanese Steakhouse,r0PIAIRRsaxpF7z9SRCT1g,5


In [15]:
#Randomly Select 200 data 
#UBCF_data=UBCF_data.sample(n=200)

In [16]:
UBCF_data['business_id']=UBCF_data['business_id'].apply(hash)

In [17]:
UBCF_data['user_id']=UBCF_data['user_id'].apply(hash)

In [18]:
business=UBCF_data[['business_id','name']]

In [19]:
business=business.rename(columns={"business_id": "business_id", "name": "title"})


In [20]:
business.head()

,business_id,title
0,985046985895203881,Enso Tea Bar
1,1786073356298238185,Turnt Up Tours
2,-1160420471496825915,Surefire Pest Control
3,7558323575540190489,Drai's After Hours
4,6842406612477306789,Musashi Japanese Steakhouse


In [21]:
Mean = UBCF_data.groupby(by="user_id",as_index=False)['stars'].mean()
Rating_avg = pd.merge(UBCF_data,Mean,on='user_id')
Rating_avg['adg_rating']=Rating_avg['stars_x']-Rating_avg['stars_y']
Rating_avg.head()

,business_id,name,user_id,stars_x,stars_y,adg_rating
0,985046985895203881,Enso Tea Bar,892498043486122473,3,4.0,-1.0
1,2188648278942301092,Infused Tea Lounge,892498043486122473,5,4.0,1.0
2,1786073356298238185,Turnt Up Tours,2113000159721549545,5,5.0,0.0
3,-1160420471496825915,Surefire Pest Control,-3472411298139621908,5,5.0,0.0
4,7558323575540190489,Drai's After Hours,-5588864271865519772,5,5.0,0.0


In [22]:
check = pd.pivot_table(Rating_avg,values='stars_x',index='user_id',columns='business_id')
check.head()

business_id,-9222041889850809589,-9221522642467167166,-9220134465678092017,-9219285732096988637,-9217078650247886585,-9216768140510465531,-9216236901169976752,-9214274581796385693,-9213885022387420556,-9212047259814140719,...,9205594944577506285,9209806749732561023,9213077406576584359,9214010492506917922,9214539254584928025,9215038036734564861,9217085367176827986,9217959426814947615,9218019321615823294,9219566919259808181
user_id,,,,,,,,,,,,,,,,,,,,,
-9223359662055357883,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-9223304194009471738,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-9222886279178806123,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-9222866561410264378,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-9222732293397718434,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:

final = pd.pivot_table(Rating_avg,values='adg_rating',index='user_id',columns='business_id')
final.head()


business_id,-9222041889850809589,-9221522642467167166,-9220134465678092017,-9219285732096988637,-9217078650247886585,-9216768140510465531,-9216236901169976752,-9214274581796385693,-9213885022387420556,-9212047259814140719,...,9205594944577506285,9209806749732561023,9213077406576584359,9214010492506917922,9214539254584928025,9215038036734564861,9217085367176827986,9217959426814947615,9218019321615823294,9219566919259808181
user_id,,,,,,,,,,,,,,,,,,,,,
-9223359662055357883,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-9223304194009471738,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-9222886279178806123,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-9222866561410264378,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-9222732293397718434,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


final df contains some lots of NaN value since every user has not seen all the movies and that’s the reason this type of matrix is called sparse matrix. 

In [24]:
# Replacing NaN by business Average
final_business = final.fillna(final.mean(axis=0))
# Since most of the business have ratings

# Replacing NaN by user Average
#final_user = final.apply(lambda row: row.fillna(row.mean()), axis=1)

In [25]:
final_business.head()

business_id,-9222041889850809589,-9221522642467167166,-9220134465678092017,-9219285732096988637,-9217078650247886585,-9216768140510465531,-9216236901169976752,-9214274581796385693,-9213885022387420556,-9212047259814140719,...,9205594944577506285,9209806749732561023,9213077406576584359,9214010492506917922,9214539254584928025,9215038036734564861,9217085367176827986,9217959426814947615,9218019321615823294,9219566919259808181
user_id,,,,,,,,,,,,,,,,,,,,,
-9223359662055357883,0.0,0.0,0.0,0.0,0.0,0.1875,-0.166667,-0.125,0.0,0.133333,...,0.75,0.466667,-0.148148,-1.0,0.0,0.0,-2.0,0.0,0.055556,0.142857
-9223304194009471738,0.0,0.0,0.0,0.0,0.0,0.1875,-0.166667,-0.125,0.0,0.133333,...,0.75,0.466667,-0.148148,-1.0,0.0,0.0,-2.0,0.0,0.055556,0.142857
-9222886279178806123,0.0,0.0,0.0,0.0,0.0,0.1875,-0.166667,-0.125,0.0,0.133333,...,0.75,0.466667,-0.148148,-1.0,0.0,0.0,-2.0,0.0,0.055556,0.142857
-9222866561410264378,0.0,0.0,0.0,0.0,0.0,0.1875,-0.166667,-0.125,0.0,0.133333,...,0.75,0.466667,-0.148148,-1.0,0.0,0.0,-2.0,0.0,0.055556,0.142857
-9222732293397718434,0.0,0.0,0.0,0.0,0.0,0.1875,-0.166667,-0.125,0.0,0.133333,...,0.75,0.466667,-0.148148,-1.0,0.0,0.0,-2.0,0.0,0.055556,0.142857


In [26]:
#final_user.head()

## calculate the similarity between the users

In [27]:
# user similarity on replacing NAN by user avg

# b = cosine_similarity(final_business)
# np.fill_diagonal(b, 0 )
# similarity_with_user = pd.DataFrame(b,index=final_user.index)
# similarity_with_user.columns=final_user.index
# similarity_with_user.head()

In [ ]:
# user similarity on replacing NAN by item(business) avg
cosine = cosine_similarity(final_business)
np.fill_diagonal(cosine, 0 )
similarity_with_business = pd.DataFrame(cosine,index=final_business.index)
similarity_with_business.columns=final_business.index
similarity_with_business.head()

## KNN
calculating the score for any item it would not be a good solution or method to look at all the other users all the time. Hence to overcome this we generate a notion of neighborhood. This includes only the set of (K) similar users for a particular user.

In [ ]:
def find_n_neighbours(df,n):
    order = np.argsort(df.values, axis=1)[:, :n]
    df = df.apply(lambda x: pd.Series(x.sort_values(ascending=False)
           .iloc[:n].index, 
          index=['top{}'.format(i) for i in range(1, n+1)]), axis=1)
    return df

In [ ]:
# top 30 neighbours for each user
sim_user_30_b = find_n_neighbours(similarity_with_business,30)
sim_user_30_b.head()

In [ ]:
def User_item_score(user,item):
    a = sim_user_30_b[sim_user_30_b.index==user].values
    b = a.squeeze().tolist()
    c = final_movie.loc[:,item]
    d = c[c.index.isin(b)]
    f = d[d.notnull()]
    avg_user = Mean.loc[Mean['user_id'] == user,'stars'].values[0]
    index = f.index.values.squeeze().tolist()
    corr = similarity_with_business.loc[user,index]
    fin = pd.concat([f, corr], axis=1)
    fin.columns = ['adg_score','correlation']
    fin['score']=fin.apply(lambda x:x['adg_score'] * x['correlation'],axis=1)
    nume = fin['score'].sum()
    deno = fin['correlation'].sum()
    final_score = avg_user + (nume/deno)
    return final_score

In [ ]:
Rating_avg = Rating_avg.astype({"business_id": str})
Movie_user = Rating_avg.groupby(by = 'user_id')['business_id'].apply(lambda x:','.join(x))

In [ ]:
def User_item_score1(user):
    Movie_seen_by_user = check.columns[check[check.index==user].notna().any()].tolist()
    a = sim_user_30_b[sim_user_30_b.index==user].values
    b = a.squeeze().tolist()
    d = Movie_user[Movie_user.index.isin(b)]
    l = ','.join(d.values)
    Movie_seen_by_similar_users = l.split(',')
    Movies_under_consideration = list(set(Movie_seen_by_similar_users)-set(list(map(str, Movie_seen_by_user))))
    Movies_under_consideration = list(map(int, Movies_under_consideration))
    score = []
    for item in Movies_under_consideration:
        c = final_business.loc[:,item]
        d = c[c.index.isin(b)]
        f = d[d.notnull()]
        avg_user = Mean.loc[Mean['user_id'] == user,'stars'].values[0]
        index = f.index.values.squeeze().tolist()
        corr = similarity_with_business.loc[user,index]
        fin = pd.concat([f, corr], axis=1)
        fin.columns = ['adg_score','correlation']
        fin['score']=fin.apply(lambda x:x['adg_score'] * x['correlation'],axis=1)
        nume = fin['score'].sum()
        deno = fin['correlation'].sum()
        final_score = avg_user + (nume/deno)
        score.append(final_score)
    data = pd.DataFrame({'business_id':Movies_under_consideration,'score':score})
    top_5_recommendation = data.sort_values(by='score',ascending=False).head(5)
    Movie_Name = top_5_recommendation.merge(business, how='inner', on='business_id')
    Movie_Names = Movie_Name.title.values.tolist()
    return Movie_Names

In [ ]:
user = int(input("Enter the user id to whom you want to recommend : "))
predicted_movies = User_item_score1(user)
print(" ")
print("The Recommendations for this user")
print("   ")
for i in predicted_movies:
    print(i)


In [ ]:
3002164458318600832	

Enter the user id to whom you want to recommend : 3002164458318600832
 
The Recommendations for this user
   
La Flor de Michoacan Ice Cream Shop
Beach Cafe
Island Flavor
Women's Specialty Care
Tahiti Village